# PYSPARK VERİ MANİPULASYONU VE MAKİNA ÖĞRENMESİ METHODU 
<font size=4,font style=arial>
<br><br>
Merhaba,<br>
Pyspark kodları ile önce veri manipulasyonu yapacağız. İki tane veri setini rastgele sayılarla oluşturup join yapıp ,aşağıdaki fonksiyonları ve metodları uygulayacağız:  <br>
1-Case when <br>
2-Distinct <br>
3-Kolon adları değiştirme <br>
4-Kolonların min,max,avg,sum,count,standard deviation ı bulma<br>
5-Group by sum,count,mean <br>
6-Filter <br>
7-Rank <br>
8-Makine Öğrenmesi methodu olan Lojistik Regresyon Analizi <br>
</font>

<font size=4,font style=arial>
Öncelikle gerekli fonksiyonları yükleyelim ve spark contexti oluşuralım
</font>

In [1]:
#round fonksiyonu  için
import pyspark.sql.functions as func
##Spark session oluşturulur
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import rand, randn,when,mean, min, max,sum,stddev,count,desc,col
from pyspark.sql.window import Window
sc =SparkContext()
sqlContext = SQLContext(sc)

<font size=4,font style=arial>
<br>
İlk veri setimizi(data frame) oluşturalım ve adı df1 olsun. 1 milyon satırlı 5 kolonlu oluşturalım. Kolonlar;2 'si uniform dağılım, 2'si normal dağılım ve bir kolonda 0,1 sayıları içerecek şekilde olsun.<br>
id:id kolonu<br>
uniform: uniform dağılımlı kolon<br>
uniform1: uniform dağılımlı kolon<br>
normal: normal dağılımlı kolon<br>
normal1: normal dağılımlı kolon<br>
Y:0,1 içeren kolon<br>
NOT:func.round oluşturduğumzu rastgele sayıları yuvarlamak içindir <br>
NOT: Sparksjobs ları http://localhost:4040/jobs/ 'den takip edebilirsiniz    
</font>

In [33]:
df1 = sqlContext.range(0, 1000000)\
     .withColumn('uniform', func.round(rand(seed=10),2))\
     .withColumn('uniform1',func.round(rand(seed=9),2))\
     .withColumn('normal',func.round(randn(seed=22),2))\
     .withColumn('normal1',func.round(randn(seed=23),2))\
     .withColumn('Y', when(rand() > 0.5, 1).otherwise(0))

<font size=4,font style=arial>
df1 satır sayısı
    </font>

In [34]:
#df1 satır sayısı
df1.count()

1000000

<font size=4,font style=arial>
İkinci veri setimiz df2 olsun.
<br>    
id=id kolunu
    <br>
uniform2: uniform dağılımlı kolon
<br>
normal2: normal dağılımlı kolon
<br>
normal3: normal dağılımlı kolon
</font>

In [35]:
df2 = sqlContext.range(0, 1000000)\
     .withColumn('uniform2', func.round(rand(seed=9),2))\
     .withColumn('normal2', func.round(randn(seed=3),2))\
     .withColumn('normal3', func.round(randn(seed=5),2))

<font size=4,font style=arial>
df2 satır sayısı
</font>

In [36]:
df2.count()

1000000

<font size=4,font style=arial>
df1 verisi için en tepedeki 5 satırı görelim
</font>

In [37]:
df1.show(5)

+---+-------+--------+------+-------+---+
| id|uniform|uniform1|normal|normal1|  Y|
+---+-------+--------+------+-------+---+
|  0|   0.41|    0.71|  1.02|    0.1|  1|
|  1|   0.73|    0.64| -0.39|  -0.36|  0|
|  2|    0.9|    0.29|  2.14|   0.28|  1|
|  3|   0.09|    0.05| -0.14|  -1.11|  0|
|  4|   0.38|     0.5|  0.44|   0.63|  1|
+---+-------+--------+------+-------+---+
only showing top 5 rows



<font size=4,font style=arial>
df2'den de bir örnek alalım. 5'tanesini görelim. (Set seed sürekli aynı örneklemin gelmesini sağlar.)
</font>

In [38]:
df2.sample(False, 0.1, seed=0).limit(5).show()

+---+--------+-------+-------+
| id|uniform2|normal2|normal3|
+---+--------+-------+-------+
| 20|    0.38|  -0.42|  -0.14|
| 33|    0.89|   -0.1|   0.33|
| 39|     0.7|   0.39|  -1.17|
| 44|    0.62|    0.2|  -1.17|
| 48|    0.73|   1.73|   1.43|
+---+--------+-------+-------+



# CASE WHEN 
<br>
<br>
<font size=4,font style=arial>
df1'e C1 adında bir kolon ekleyelim.Case when 'li kurallarımız;
<br>
df1.normal <= 0.1 ise A <br>
df1.normal > 0.1 ve df1.normal =< 0.7 ise 'B' <br> 
df1.normal > 0.7 ve df1.normal =< 1  ise 'C'  
df1.normal > 1 ve df1.normal<1.3 ise 'D'  <br> 
df1.normal >= 1.3 ise 'E'   
</font>

In [39]:
#parantezler önemli yoksa error mesajı alınır
df1=df1.withColumn('C1', when(df1.normal<=0.1, 'A')\
                    .when((df1.normal>0.1) & (df1.normal<=0.7), 'B')\
                    .when((df1.normal>0.7) & (df1.normal<=1), 'C')\
                    .when((df1.normal>1) & (df1.normal<1.3), 'D')\
                    .otherwise('E')) 

# DISTINCT
<br> 
<br> 
<font size=4,font style=arial>
Oluşturdumuz C1 kolonuna distinct ile bir göz atalım   
</font>

In [40]:
df1.select('C1').distinct().show()

+---+
| C1|
+---+
|  E|
|  B|
|  D|
|  C|
|  A|
+---+



<font size=4,font style=arial>
df1 veri setimize bir göz atalım
</font>    

In [41]:
df1.show(10)

+---+-------+--------+------+-------+---+---+
| id|uniform|uniform1|normal|normal1|  Y| C1|
+---+-------+--------+------+-------+---+---+
|  0|   0.41|    0.71|  1.02|    0.1|  1|  D|
|  1|   0.73|    0.64| -0.39|  -0.36|  0|  A|
|  2|    0.9|    0.29|  2.14|   0.28|  1|  E|
|  3|   0.09|    0.05| -0.14|  -1.11|  0|  A|
|  4|   0.38|     0.5|  0.44|   0.63|  1|  B|
|  5|   0.56|    0.13| -0.66|  -0.89|  0|  A|
|  6|    0.5|     0.8|  2.61|   0.06|  1|  E|
|  7|   0.21|     0.0|  2.47|  -0.55|  0|  E|
|  8|   0.96|    0.92|  0.01|   0.83|  1|  A|
|  9|   0.74|    0.85|  0.59|   1.74|  1|  B|
+---+-------+--------+------+-------+---+---+
only showing top 10 rows



<font size=4,font style=arial>
Benzer bir çalışmayı df2 içinde yapalım.Kolon adı C2 olsun. 
<br>
df2.normal2 <= 0.3 ise 'W'
<br>
df2.normal2 > 0.3 ise 'Z' 
</font>

In [42]:
df2=df2.withColumn('C2', when(df2.normal2<=0.3, 'W')\
                     .otherwise('Z')) 

<font size=4,font style=arial>
df2'e göz atalım
</font>

In [43]:
df2.show(10)

+---+--------+-------+-------+---+
| id|uniform2|normal2|normal3| C2|
+---+--------+-------+-------+---+
|  0|    0.71|  -0.92|  -0.82|  W|
|  1|    0.64|   0.25|   0.16|  W|
|  2|    0.29|   0.71|    1.8|  Z|
|  3|    0.05|   -0.8|  -0.56|  W|
|  4|     0.5|   0.31|  -0.16|  Z|
|  5|    0.13|   0.03|   2.58|  W|
|  6|     0.8|  -0.16|   0.01|  W|
|  7|     0.0|   0.08|  -0.09|  W|
|  8|    0.92|  -0.46|   0.04|  W|
|  9|    0.85|  -1.59|   0.46|  W|
+---+--------+-------+-------+---+
only showing top 10 rows



# JOIN
<br>
<br>
<font size=4,font style=arial>
df1 ile df2 join yapalım ekstradan gelen id kolonunu drop edelim
</font>

In [44]:
df = df1.join(df2, df1.id == df2.id).drop(df2.id)

<font size=4,font style=arial>
df kontrol edelim
</font>

In [45]:
df.printSchema()

root
 |-- id: long (nullable = false)
 |-- uniform: double (nullable = true)
 |-- uniform1: double (nullable = true)
 |-- normal: double (nullable = true)
 |-- normal1: double (nullable = true)
 |-- Y: integer (nullable = false)
 |-- C1: string (nullable = false)
 |-- uniform2: double (nullable = true)
 |-- normal2: double (nullable = true)
 |-- normal3: double (nullable = true)
 |-- C2: string (nullable = false)



<br>
<br>
<font size=4,font style=arial>
df'in top 10 kontrolü
</font>

In [46]:
df.show(10)

+----+-------+--------+------+-------+---+---+--------+-------+-------+---+
|  id|uniform|uniform1|normal|normal1|  Y| C1|uniform2|normal2|normal3| C2|
+----+-------+--------+------+-------+---+---+--------+-------+-------+---+
|  26|   0.82|     0.5| -0.57|  -0.11|  1|  A|     0.5|    0.4|   0.15|  Z|
|  29|   0.21|    0.49|  0.82|   -1.3|  1|  C|    0.49|  -1.23|   0.24|  W|
| 474|   0.43|    0.49| -1.02|   0.21|  1|  A|    0.49|   0.33|   1.73|  Z|
| 964|   0.19|    0.88|  0.96|  -0.19|  0|  C|    0.88|  -1.85|   0.79|  W|
|1677|   0.95|    0.22|  1.17|  -2.95|  1|  D|    0.22|   2.51|   1.46|  Z|
|1697|   0.17|    0.71| -1.27|   0.06|  0|  A|    0.71|   1.47|  -0.86|  Z|
|1806|    0.7|    0.69|  1.17|    0.1|  0|  D|    0.69|   0.11|   0.74|  W|
|1950|   0.21|    0.45| -1.62|  -0.84|  1|  A|    0.45|  -0.12|   1.34|  W|
|2040|   0.33|    0.61|   1.3|    1.2|  1|  E|    0.61|  -0.57|   0.81|  W|
|2214|   0.88|    0.73|  0.47|  -0.73|  0|  B|    0.73|   0.88|  -0.57|  Z|
+----+------

# YENI KOLON EKLEME
<br>
<br>
<font size=4,font style=arial>
df'de bulunan uniform1 ve normal kolonlarını kullanarak yeni bir kolon oluşuralım şu an bu kolunu dataframe'e yazmayacağız
</font>

In [47]:
df.withColumn('yeni_kolon',3*df.uniform1-3*df.normal ).show(5)

+----+-------+--------+------+-------+---+---+--------+-------+-------+---+--------------------+
|  id|uniform|uniform1|normal|normal1|  Y| C1|uniform2|normal2|normal3| C2|          yeni_kolon|
+----+-------+--------+------+-------+---+---+--------+-------+-------+---+--------------------+
|  26|   0.82|     0.5| -0.57|  -0.11|  1|  A|     0.5|    0.4|   0.15|  Z|                3.21|
|  29|   0.21|    0.49|  0.82|   -1.3|  1|  C|    0.49|  -1.23|   0.24|  W|               -0.99|
| 474|   0.43|    0.49| -1.02|   0.21|  1|  A|    0.49|   0.33|   1.73|  Z|                4.53|
| 964|   0.19|    0.88|  0.96|  -0.19|  0|  C|    0.88|  -1.85|   0.79|  W|-0.23999999999999977|
|1677|   0.95|    0.22|  1.17|  -2.95|  1|  D|    0.22|   2.51|   1.46|  Z| -2.8499999999999996|
+----+-------+--------+------+-------+---+---+--------+-------+-------+---+--------------------+
only showing top 5 rows



# KOLON ADLARINI DEĞİŞTİRME
<br>
<br>
<font size=4,font style=arial>
Yeni kolon adlarımız U1,U2,U3,N1,N2,N3,N4'e çevirelim.  
</font>

In [48]:
df = df.withColumnRenamed("uniform", "U1")\
       .withColumnRenamed("uniform1", "U2")\
       .withColumnRenamed("uniform2", "U3")\
       .withColumnRenamed("normal", "N1")\
       .withColumnRenamed("normal1", "N2")\
       .withColumnRenamed("normal2", "N3")\
       .withColumnRenamed("normal3", "N4")
df.printSchema()

root
 |-- id: long (nullable = false)
 |-- U1: double (nullable = true)
 |-- U2: double (nullable = true)
 |-- N1: double (nullable = true)
 |-- N2: double (nullable = true)
 |-- Y: integer (nullable = false)
 |-- C1: string (nullable = false)
 |-- U3: double (nullable = true)
 |-- N3: double (nullable = true)
 |-- N4: double (nullable = true)
 |-- C2: string (nullable = false)



# KOLONUN MİN,MAX,AVG,SUM,COUNT,STANDARD DEVIATION'NI BULMA
<br>
<font size=4,font style=arial>
N1 kolonu için bulalım.Method 1 aggregate fonksiyonunu, Method 2 select'i kullanıyor. İsterseniz de withColumnRenamed fonksiyonu kullanarak kolona yeni ad verilebilir. func.round ilede round yapabilirsiniz.
</font>

In [49]:
#1.method
df.agg(min("N1"),max("N1"),mean("N1"),func.round(sum("N1"),2),stddev("N1"),count("N1"))\
.withColumnRenamed("min(N1)", "min").show()

+-----+-------+--------------------+-----------------+------------------+---------+
|  min|max(N1)|             avg(N1)|round(sum(N1), 2)|   stddev_samp(N1)|count(N1)|
+-----+-------+--------------------+-----------------+------------------+---------+
|-4.59|   4.79|0.001570150000000...|          1570.15|0.9998111433422301|  1000000|
+-----+-------+--------------------+-----------------+------------------+---------+



In [50]:
#2.method
df.select([min('N1'), max('N1'),mean('N1'),sum('N1'),stddev('N1'),count('N1')]).show()

+-------+-------+--------------------+------------------+------------------+---------+
|min(N1)|max(N1)|             avg(N1)|           sum(N1)|   stddev_samp(N1)|count(N1)|
+-------+-------+--------------------+------------------+------------------+---------+
|  -4.59|   4.79|0.001570150000000...|1570.1500000000003|0.9998111433422301|  1000000|
+-------+-------+--------------------+------------------+------------------+---------+



# GROUP BY 
<br>
<font size=4,font style=arial>
Yukarıda buldugumuz değerleri C1 ve C2 için group by şeklinde bulalım
</font>

In [51]:
df.groupby('C1','C2').agg(min("N1"),max("N1"),mean("N1"),func.round(sum("N1"),2),stddev("N1"),count("N1"))\
.withColumnRenamed("min(N1)", "min").show() 

+---+---+-----+-------+-------------------+-----------------+-------------------+---------+
| C1| C2|  min|max(N1)|            avg(N1)|round(sum(N1), 2)|    stddev_samp(N1)|count(N1)|
+---+---+-----+-------+-------------------+-----------------+-------------------+---------+
|  C|  W| 0.71|    1.0| 0.8484908037288995|         43779.58|0.08629206806569498|    51597|
|  B|  Z| 0.11|    0.7| 0.3936189467603737|         32708.16|0.17211378663207755|    83096|
|  A|  W|-4.43|    0.1|-0.7324595520390188|       -245689.64| 0.6228749728531819|   335431|
|  D|  Z| 1.01|   1.29|   1.14267522855899|         26247.25|0.08344406026535031|    22970|
|  E|  W|  1.3|   4.63|  1.766692768309535|        107393.72| 0.4095826616247165|    60788|
|  A|  Z|-4.59|    0.1|-0.7303695132149598|       -150219.48| 0.6207203777615165|   205676|
|  C|  Z| 0.71|    1.0|  0.847848909212156|         26699.61|0.08624057518841813|    31491|
|  B|  W| 0.11|    0.7| 0.3923364721958172|         52830.46|0.17179409633061266

# FILTER
<br>
<br>
<font size=4,font style=arial>
FILTER fonksiyonu aşağıda ki şekilde hesaplanır. 
(C1=A) ve (U1 0.9'a eşit olmasın) ve (Y 0'a eşit olsun) ve (N1 0'dan büyük olsun) 
</font>

In [52]:
df.filter('C1="A" and U1 != 0.9 and Y=0 and N1>0').show(10)

+-----+----+----+----+-----+---+---+----+-----+-----+---+
|   id|  U1|  U2|  N1|   N2|  Y| C1|  U3|   N3|   N4| C2|
+-----+----+----+----+-----+---+---+----+-----+-----+---+
| 3091|0.42|0.18|0.03|-0.22|  0|  A|0.18| -0.5|-0.23|  W|
|11567|0.26|0.39|0.09| 0.64|  0|  A|0.39|-0.49|  2.0|  W|
|15237|0.07|0.84|0.05|-1.13|  0|  A|0.84|-0.39|-1.68|  W|
|31762|0.92|0.48|0.07| -1.0|  0|  A|0.48|-1.31|-0.39|  W|
|32912|0.12|0.95|0.05| 0.64|  0|  A|0.95|-0.12| 0.34|  W|
|38287|0.21|0.53| 0.1|-0.61|  0|  A|0.53| 0.79|-0.26|  Z|
|38996|0.64|0.83|0.06| 1.26|  0|  A|0.83| 0.95|-0.13|  Z|
|40436|0.81|0.42|0.02|  2.5|  0|  A|0.42| 0.26| 1.96|  W|
|49586|0.74|0.08|0.02| 1.27|  0|  A|0.08|-0.87| -1.5|  W|
|81293|0.42|0.63|0.09| 0.07|  0|  A|0.63| 0.08|-1.58|  W|
+-----+----+----+----+-----+---+---+----+-----+-----+---+
only showing top 10 rows



<font size=4,font style=arial>
FİLTER 'dan sonra min,max,mean,sum,stddev'ını bulalım
</font>

In [53]:
df.filter('C1="A" and U1 != 0.9 and Y=0 and N1>0')\
         .agg(min("N1"),max("N1"),mean("N1"),func.round(sum("N1"),2),stddev("N1"),count("N1")).show()

+-------+-------+-------------------+-----------------+--------------------+---------+
|min(N1)|max(N1)|            avg(N1)|round(sum(N1), 2)|     stddev_samp(N1)|count(N1)|
+-------+-------+-------------------+-----------------+--------------------+---------+
|   0.01|    0.1|0.05488298898421269|          1081.14|0.028713581417572524|    19699|
+-------+-------+-------------------+-----------------+--------------------+---------+



<font size=4,font style=arial>
Çapraz tablolama
</font>

In [54]:
df.crosstab('C1', 'C2').show()

+-----+------+------+
|C1_C2|     W|     Z|
+-----+------+------+
|    E| 60788| 36924|
|    A|335431|205676|
|    B|134656| 83096|
|    C| 51597| 31491|
|    D| 37371| 22970|
+-----+------+------+



<font size=4,font style=arial>
distinct değerleri görme
</font>

In [55]:
df.select('C1','C2').dropDuplicates().show()

+---+---+
| C1| C2|
+---+---+
|  C|  W|
|  B|  Z|
|  A|  W|
|  D|  Z|
|  E|  W|
|  A|  Z|
|  C|  Z|
|  B|  W|
|  D|  W|
|  E|  Z|
+---+---+



# ORDER BY-SORT DESCENDING
<br>
<br>
<font size=4,font style=arial>
C1 ve C2 değişkenine göre count alıp büyükten küçüğe doğru sıralayım
</font>

In [56]:
df.groupBy('C1','C2').count().orderBy('count',ascending=False).show()

+---+---+------+
| C1| C2| count|
+---+---+------+
|  A|  W|335431|
|  A|  Z|205676|
|  B|  W|134656|
|  B|  Z| 83096|
|  E|  W| 60788|
|  C|  W| 51597|
|  D|  W| 37371|
|  E|  Z| 36924|
|  C|  Z| 31491|
|  D|  Z| 22970|
+---+---+------+



# RANK
<br>
<br>
<font size=4,font style=arial>
C1 ve C2 değişkenine göre N1 büyükten küçüğe sıralanacak şekilde rank numarası ata. Herbir grupdan top 2'yi seç
</font>

In [57]:
df.withColumn("row_number", func.row_number().over(Window.partitionBy("C1","C2").orderBy(desc("N1"))))\
  .filter(col('row_number') <= 2).show()      

+------+----+----+----+-----+---+---+----+-----+-----+---+----------+
|    id|  U1|  U2|  N1|   N2|  Y| C1|  U3|   N3|   N4| C2|row_number|
+------+----+----+----+-----+---+---+----+-----+-----+---+----------+
| 30421|0.18|0.74| 1.0| 0.52|  0|  C|0.74|-0.88| 0.68|  W|         1|
| 33209| 0.7| 0.4| 1.0|-0.74|  0|  C| 0.4| 0.24| 1.42|  W|         2|
| 10422|0.29|0.64| 0.7| 1.04|  0|  B|0.64| 0.89| 0.28|  Z|         1|
|770069|0.49|0.27| 0.7| 0.98|  1|  B|0.27| 1.04|-1.09|  Z|         2|
|123425|0.11|0.65| 0.1| 0.95|  0|  A|0.65|-0.61| 0.52|  W|         1|
|161554|0.11|0.36| 0.1|-0.29|  1|  A|0.36|-0.37| 0.83|  W|         2|
|324942|0.82|0.48|1.29|-0.75|  1|  D|0.48| 2.71| -0.9|  Z|         1|
|547847| 0.4|0.09|1.29|-0.65|  0|  D|0.09| 0.58| 0.71|  Z|         2|
|145352|0.15|0.49|4.63| 0.14|  0|  E|0.49|-0.89|-0.02|  W|         1|
|988632|0.02|0.96|4.54| 0.27|  0|  E|0.96|-1.17|-0.14|  W|         2|
| 38287|0.21|0.53| 0.1|-0.61|  0|  A|0.53| 0.79|-0.26|  Z|         1|
| 98559|0.54|0.98| 0

# MACHINE LEARNING
<br>
<br>
<font size=4,font style=arial>
Lojistik regresyon uygulamadan önce gerekli paketleri yükleyelim. 
</font>

In [58]:
import numpy
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import BinaryLogisticRegressionSummary,LogisticRegression
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,MulticlassClassificationEvaluator) 

<br>
<font size=4,font style=arial>
Bağımlı değişkenimiz Y ve bağımsız değişkenlerimiz U1,U2,U3,N1,N2,N3,N4,C1,C2 olmak üzere Lojistik regresyon analizi yapalım. Spark model için bir features (bağımsız değişkenlerin oluşturduğu sparse matris) ve label(bağımlı değişken) vektörlerinin oluşturulması gerekiyor. R formula bunu oluşturmaktadır.
</font>

In [59]:
formula = RFormula(formula="Y ~ U1+U2+U3+N1+N2+N3+N4+C1+C2")
output = formula.fit(df).transform(df)

<font size=4,font style=arial>
Modelimiz için gerekli olan features(yer kaplamaması ve işlem kolaylığı açısından oluşturulan sparse matris)  ve label kolonu oluştu. Aşağıda da çıktısı var.
</font>    

In [60]:
output.show(5,truncate=False) 

+----+----+----+-----+-----+---+---+----+-----+----+---+----------------------------------------------------------+-----+
|id  |U1  |U2  |N1   |N2   |Y  |C1 |U3  |N3   |N4  |C2 |features                                                  |label|
+----+----+----+-----+-----+---+---+----+-----+----+---+----------------------------------------------------------+-----+
|26  |0.82|0.5 |-0.57|-0.11|1  |A  |0.5 |0.4  |0.15|Z  |[0.82,0.5,0.5,-0.57,-0.11,0.4,0.15,1.0,0.0,0.0,0.0,0.0]   |1.0  |
|29  |0.21|0.49|0.82 |-1.3 |1  |C  |0.49|-1.23|0.24|W  |[0.21,0.49,0.49,0.82,-1.3,-1.23,0.24,0.0,0.0,0.0,1.0,1.0] |1.0  |
|474 |0.43|0.49|-1.02|0.21 |1  |A  |0.49|0.33 |1.73|Z  |[0.43,0.49,0.49,-1.02,0.21,0.33,1.73,1.0,0.0,0.0,0.0,0.0] |1.0  |
|964 |0.19|0.88|0.96 |-0.19|0  |C  |0.88|-1.85|0.79|W  |[0.19,0.88,0.88,0.96,-0.19,-1.85,0.79,0.0,0.0,0.0,1.0,1.0]|0.0  |
|1677|0.95|0.22|1.17 |-2.95|1  |D  |0.22|2.51 |1.46|Z  |[0.95,0.22,0.22,1.17,-2.95,2.51,1.46,0.0,0.0,0.0,0.0,0.0] |1.0  |
+----+----+----+-----+--

<font size=4,font style=arial>
Model için sadece features ve label kolonunu alacağız.
</font>

In [61]:
df_log=output.select([c for c in output.columns if c not in {'id','U1','U2','N1','N2','Y','C1','U3','N3','N4','C2'}])

In [62]:
df_log

DataFrame[features: vector, label: double]

In [63]:
df_log.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.82,0.5,0.5,-0....|  1.0|
|[0.21,0.49,0.49,0...|  1.0|
|[0.43,0.49,0.49,-...|  1.0|
|[0.19,0.88,0.88,0...|  0.0|
|[0.95,0.22,0.22,1...|  1.0|
+--------------------+-----+
only showing top 5 rows



<font size=4,font style=arial>
Veri setimizi %70 ve %30 şeklinde ikiye bölüp %70'de modeli kurup %30'da test edelim.
</font>

In [64]:
lr_train,lr_test=df_log.randomSplit([0.7,0.3])

<font size=4,font style=arial>
Modelimizi uygulayalım
</font>

In [65]:
final_model=LogisticRegression()
fit_final_model=final_model.fit(lr_train)

<font size=4,font style=arial>
train setine uyguladığımız modeli test edelim
</font>

In [66]:
predictions_and_labels=fit_final_model.evaluate(lr_test)

<font size=4,font style=arial>
label gerçekleşen ve prediction da tahmin olmak üzere aşağıda ki şekildedir
</font>

In [67]:
predictions_and_labels.predictions.show(100,truncate=False)

+------------------------------------------------------------+-----+----------------------------------------------+----------------------------------------+----------+
|features                                                    |label|rawPrediction                                 |probability                             |prediction|
+------------------------------------------------------------+-----+----------------------------------------------+----------------------------------------+----------+
|(12,[0,3,4,5,6,7],[0.18,-0.31,-0.66,0.5,-0.59,1.0])         |1.0  |[0.003917210558980503,-0.003917210558980503]  |[0.5009793013875015,0.4990206986124985] |0.0       |
|(12,[0,3,4,5,6,8],[0.05,0.26,-1.34,1.07,-0.32,1.0])         |0.0  |[0.0014529494943975717,-0.0014529494943975717]|[0.500363237309698,0.49963676269030194] |0.0       |
|[0.0,0.06,0.06,0.95,-0.52,0.41,0.28,0.0,0.0,0.0,1.0,0.0]    |0.0  |[0.007572348874965967,-0.007572348874965967]  |[0.5018930781729178,0.4981069218270821] |0.0 

<font size=4,font style=arial>
Roc eğrisinin altında ki alanı rakam olarak görelim. 1'e yakın bir değer iyi bir değerdir. Veri seti manual oluşturulduğundan aşağıda ki şekilde bir değer çıkmıştır.
</font>

In [68]:
my_eval=BinaryClassificationEvaluator()
my_final_roc=my_eval.evaluate(predictions_and_labels.predictions)
my_final_roc

0.49920422881298465